<a href="https://colab.research.google.com/github/aravind-gk/Deep-Learning-for-NLP-Assignments/blob/main/Assignment-5/Transformer_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import csv
import string
import re
from tensorflow.keras import models,layers,Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [49]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet

In [51]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Fetch data

In [23]:
df = pd.read_csv("/content/drive/MyDrive/Datasets/DL-NLP-A5/IMDB.csv")

Clean the data

In [53]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [54]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop and i.strip().lower().isalpha():
            final_text.append(i.strip().lower())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text
#Apply function on review column
df['review']=df['review'].apply(denoise_text)

Prepare the data

In [64]:
df_train = df.iloc[:25000]
df_test = df.iloc[25000:]

In [65]:
tk = keras.preprocessing.text.Tokenizer(filters='"#$*+/:;<=>@[\\]^_{|}~\t\n')
tk.fit_on_texts(df_train.review)

def prep_text(txt, tk, max_seq_len):
    text_seqs = tk.texts_to_sequences(txt)
    return tf.keras.preprocessing.sequence.pad_sequences(text_seqs, maxlen=max_seq_len)

data_train = prep_text(df_train.review, tk,60)
data_test = prep_text(df_test.review, tk,60)

size_of_vocabulary=len(tk.word_index)+1
size_of_vocabulary

63152

In [66]:
data_train = keras.preprocessing.sequence.pad_sequences(data_train, maxlen=maxlen)
data_test = keras.preprocessing.sequence.pad_sequences(data_test, maxlen=maxlen)
dic = {'positive':1, 'negative':0}
df_train['sentiment'] = df_train['sentiment'].replace(dic)
df_test['sentiment'] = df_test['sentiment'].replace(dic)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Load embeddings

In [67]:
## Load pretrained datasets
# load the whole embedding into memory
def load_Mat(addrs,size_of_vocabulary,tk):
    embeddings_index = dict()
    f = open(addrs)
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

    f.close()
    print('Loaded %s word vectors.' % len(embeddings_index))
    
    # create a weight matrix for words in training docs
    embedding_matrix = np.zeros((size_of_vocabulary, 300))
    c=0
    for word, i in tk.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            c+=1
        else:
            #print(word)
            pass
    print('No. of out of vocab word in train set= %s'%(size_of_vocabulary-c))
    
    return embedding_matrix

glove=load_Mat("/content/drive/MyDrive/Embeddings/glove.6B.300d.txt",size_of_vocabulary,tk)

Loaded 400000 word vectors.
No. of out of vocab word in train set= 9780


# Reproducing previous results

In [68]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


## Without positional embeddings

In [69]:
# without positional embeddings
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, weight_matrix, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim, weights = [weight_matrix], trainable = False)
        # self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        # maxlen = tf.shape(x)[-1]
        # positions = tf.range(start=0, limit=maxlen, delta=1)
        # positions = self.pos_emb(positions)
        x = self.token_emb(x)
        # return x + positions
        return x

In [70]:

embed_dim = 300  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 200
vocab_size = size_of_vocabulary

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, glove, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)


In [40]:
# using glove, without position embeddings
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    data_train, df_train.sentiment.values, batch_size=32, epochs=2, validation_data=(data_test, df_test.sentiment.values)
)

Epoch 1/2
782/782 [==============================] - 55s 68ms/step - loss: 0.5848 - accuracy: 0.6735 - val_loss: 0.4923 - val_accuracy: 0.7604
Epoch 2/2
782/782 [==============================] - 54s 69ms/step - loss: 0.4950 - accuracy: 0.7595 - val_loss: 0.4737 - val_accuracy: 0.7739


In [71]:
# using glove, without position embeddings, after cleaning
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    data_train, df_train.sentiment.values, batch_size=32, epochs=5, validation_data=(data_test, df_test.sentiment.values)
)

Epoch 1/5
782/782 [==============================] - 56s 69ms/step - loss: 0.4678 - accuracy: 0.7831 - val_loss: 0.4658 - val_accuracy: 0.7902
Epoch 2/5
782/782 [==============================] - 53s 68ms/step - loss: 0.4144 - accuracy: 0.8181 - val_loss: 0.4051 - val_accuracy: 0.8176
Epoch 3/5
782/782 [==============================] - 53s 68ms/step - loss: 0.3956 - accuracy: 0.8286 - val_loss: 0.3982 - val_accuracy: 0.8179
Epoch 4/5
782/782 [==============================] - 53s 68ms/step - loss: 0.3809 - accuracy: 0.8336 - val_loss: 0.3801 - val_accuracy: 0.8300
Epoch 5/5
782/782 [==============================] - 53s 68ms/step - loss: 0.3733 - accuracy: 0.8376 - val_loss: 0.3836 - val_accuracy: 0.8264


## With positional embeddings

In [72]:
# with positional embeddings
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, weight_matrix, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim, weights = [weight_matrix], trainable = False)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
        return x

In [73]:

embed_dim = 300  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 200
vocab_size = size_of_vocabulary

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, glove, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)


In [43]:
# using glove, with position embeddings
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    data_train, df_train.sentiment.values, batch_size=32, epochs=2, validation_data=(data_test, df_test.sentiment.values)
)

Epoch 1/2
782/782 [==============================] - 62s 78ms/step - loss: 0.5518 - accuracy: 0.7135 - val_loss: 0.4758 - val_accuracy: 0.7701
Epoch 2/2
782/782 [==============================] - 60s 77ms/step - loss: 0.4862 - accuracy: 0.7682 - val_loss: 0.5021 - val_accuracy: 0.7330


In [44]:
history = model.fit(
    data_train, df_train.sentiment.values, batch_size=32, epochs=10, validation_data=(data_test, df_test.sentiment.values)
)

Epoch 1/10
782/782 [==============================] - 60s 77ms/step - loss: 0.4635 - accuracy: 0.7794 - val_loss: 0.4543 - val_accuracy: 0.7884
Epoch 2/10
782/782 [==============================] - 60s 76ms/step - loss: 0.4490 - accuracy: 0.7895 - val_loss: 0.4585 - val_accuracy: 0.7833
Epoch 3/10
782/782 [==============================] - 60s 76ms/step - loss: 0.4363 - accuracy: 0.7960 - val_loss: 0.4447 - val_accuracy: 0.7910
Epoch 4/10
782/782 [==============================] - 60s 76ms/step - loss: 0.4259 - accuracy: 0.7996 - val_loss: 0.4621 - val_accuracy: 0.7800
Epoch 5/10
782/782 [==============================] - 60s 77ms/step - loss: 0.4172 - accuracy: 0.8072 - val_loss: 0.4551 - val_accuracy: 0.7854
Epoch 6/10
782/782 [==============================] - 60s 76ms/step - loss: 0.4104 - accuracy: 0.8079 - val_loss: 0.4485 - val_accuracy: 0.7859
Epoch 7/10
782/782 [==============================] - 60s 76ms/step - loss: 0.4025 - accuracy: 0.8124 - val_loss: 0.4522 - val_accuracy:

In [74]:
# using glove, with position embeddings, cleaned data
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    data_train, df_train.sentiment.values, batch_size=32, epochs=10, validation_data=(data_test, df_test.sentiment.values)
)

Epoch 1/10
782/782 [==============================] - 62s 77ms/step - loss: 0.4680 - accuracy: 0.7722 - val_loss: 0.4313 - val_accuracy: 0.7856
Epoch 2/10
782/782 [==============================] - 60s 76ms/step - loss: 0.4028 - accuracy: 0.8209 - val_loss: 0.3826 - val_accuracy: 0.8256
Epoch 3/10
782/782 [==============================] - 60s 76ms/step - loss: 0.3861 - accuracy: 0.8282 - val_loss: 0.3844 - val_accuracy: 0.8295
Epoch 4/10
782/782 [==============================] - 59s 76ms/step - loss: 0.3764 - accuracy: 0.8337 - val_loss: 0.3761 - val_accuracy: 0.8267
Epoch 5/10
782/782 [==============================] - 60s 76ms/step - loss: 0.3679 - accuracy: 0.8377 - val_loss: 0.3774 - val_accuracy: 0.8285
Epoch 6/10
782/782 [==============================] - 60s 77ms/step - loss: 0.3563 - accuracy: 0.8430 - val_loss: 0.3813 - val_accuracy: 0.8292
Epoch 7/10
782/782 [==============================] - 60s 77ms/step - loss: 0.3530 - accuracy: 0.8431 - val_loss: 0.3811 - val_accuracy: